In [37]:
# 1. 필요한 라이브러리 설치
!pip install beautifulsoup4 requests

import requests
from bs4 import BeautifulSoup
import sqlite3
import json

# 2. SQLite 데이터베이스 및 테이블 생성
conn = sqlite3.connect('recipes.db')
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS recipes (id INTEGER PRIMARY KEY AUTOINCREMENT, title TEXT)')
cur.execute('CREATE TABLE IF NOT EXISTS ingredients (id INTEGER PRIMARY KEY AUTOINCREMENT, recipe_id INTEGER, ingredient TEXT)')
cur.execute('CREATE TABLE IF NOT EXISTS steps (id INTEGER PRIMARY KEY AUTOINCREMENT, recipe_id INTEGER, step_order INTEGER, description TEXT)')
conn.commit()

# 3. 레시피 크롤링 함수
def get_recipe_info(food_name):
    search_url = f"https://www.10000recipe.com/recipe/list.html?q={food_name}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    link = soup.find('a', class_='common_sp_link')
    if not link:
        print("레시피를 찾을 수 없습니다.")
        return None

    recipe_id = link['href'].split('/')[-1]
    detail_url = f"https://www.10000recipe.com/recipe/{recipe_id}"
    detail_response = requests.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
    json_ld = detail_soup.find('script', type='application/ld+json')
    if not json_ld:
        print("레시피 정보를 찾을 수 없습니다.")
        return None

    data = json.loads(json_ld.string)
    title = data.get('name', '제목 없음')
    ingredients = data.get('recipeIngredient', [])
    steps = [step['text'] for step in data.get('recipeInstructions', [])]
    return {
        'title': title,
        'ingredients': ingredients,
        'steps': steps
    }

# 4. 크롤링 → DB 저장 함수
def crawl_and_save(food_name):
    data = get_recipe_info(food_name)
    if not data:
        print("데이터가 없습니다.")
        return

    # 레시피 제목 저장
    cur.execute('INSERT INTO recipes (title) VALUES (?)', (data['title'],))
    recipe_id = cur.lastrowid

    # 재료 저장
    for ing in data['ingredients']:
        cur.execute('INSERT INTO ingredients (recipe_id, ingredient) VALUES (?, ?)', (recipe_id, ing))

    # 순서 저장
    for idx, step in enumerate(data['steps'], 1):
        cur.execute('INSERT INTO steps (recipe_id, step_order, description) VALUES (?, ?, ?)', (recipe_id, idx, step))

    conn.commit()
    print(f"'{data['title']}' 레시피가 DB에 저장되었습니다.")

# 5. 사용 예시
crawl_and_save("콩나물국")  # 원하는 음식명 입력

# 6. DB에서 저장된 레시피 확인
for row in cur.execute('SELECT * FROM recipes'):
    print(row)

conn.close()


'국물이 너무 개운해요~맑은콩나물국 맛있게 끓이는법(김진옥요리가좋다)' 레시피가 DB에 저장되었습니다.
(4, '감자전')
(5, '감자탕')
(6, '제육볶음 만드는법 # 기사식당st 레시피')
(7, '매콤 된장찌개, 채소와 두부만 넣은 초간단 초스피드 된장찌개')
(8, '실패 없는 백종원 레시피 :: 소고기뭇국')
(9, '대박 맛집 김치찌개')
(10, '국물이 너무 개운해요~맑은콩나물국 맛있게 끓이는법(김진옥요리가좋다)')


In [39]:
import sqlite3

# DB 연결
conn = sqlite3.connect('recipes.db')
cur = conn.cursor()

# 1. 저장된 모든 레시피 목록 출력
print("=== 저장된 레시피 목록 ===")
cur.execute('SELECT id, title FROM recipes')
recipes = cur.fetchall()
for recipe in recipes:
    print(f"ID: {recipe[0]}, 제목: {recipe[1]}")

# 2. 특정 레시피의 상세 정보 출력 함수
def show_recipe_detail(recipe_id):
    # 제목
    cur.execute('SELECT title FROM recipes WHERE id=?', (recipe_id,))
    title = cur.fetchone()
    if not title:
        print("해당 레시피가 없습니다.")
        return
    print(f"\n=== [{title[0]}] ===")

    # 재료
    print("\n[재료]")
    cur.execute('SELECT ingredient FROM ingredients WHERE recipe_id=?', (recipe_id,))
    ingredients = cur.fetchall()
    for ing in ingredients:
        print(f"- {ing[0]}")

    # 조리 순서
    print("\n[조리 순서]")
    cur.execute('SELECT step_order, description FROM steps WHERE recipe_id=? ORDER BY step_order', (recipe_id,))
    steps = cur.fetchall()
    for step in steps:
        print(f"{step[0]}. {step[1]}")

# 3. 예시: 첫 번째 레시피의 상세 정보 출력
if recipes:
    show_recipe_detail(recipes[6][0])

conn.close()

=== 저장된 레시피 목록 ===
ID: 4, 제목: 감자전
ID: 5, 제목: 감자탕
ID: 6, 제목: 제육볶음 만드는법 # 기사식당st 레시피
ID: 7, 제목: 매콤 된장찌개, 채소와 두부만 넣은 초간단 초스피드 된장찌개
ID: 8, 제목: 실패 없는 백종원 레시피 :: 소고기뭇국
ID: 9, 제목: 대박 맛집 김치찌개
ID: 10, 제목: 국물이 너무 개운해요~맑은콩나물국 맛있게 끓이는법(김진옥요리가좋다)

=== [국물이 너무 개운해요~맑은콩나물국 맛있게 끓이는법(김진옥요리가좋다)] ===

[재료]
- 콩나물 4줌
- 다진파 5T
- 멸치다시마육수 500ml
- 다진마늘 1/2T
- 국간장 1T
- 멸치액젓 1T
- 소금 1/3T

[조리 순서]
1. 콩나물은 누런부분과 껍질부분 제거하고 두세번 세척후 소쿠리에 받쳐줍니다. 미리 멸치다시마육수 500ml도 준비해주세요~
2. 냄비에 씻은 콩나물을 넣고 멸치다시마육수 500ml를 넣은후 물 700ml를 넣고요~
3. 센불로 켠후 끓으면 중약불로 줄여 5분동안 끓여줍니다.
4. 5분경과후~ 다진마늘1/2T. 다진파5T. 국간장1T. 멸치액젓1T를 넣은후~ 한번 바글 끓인후 간을 보셔서 싱거우면 소금간을 추가하세요~ 저는 소금1/3T정도 넣었습니다.


In [40]:
import sqlite3

# 1. DB 연결
conn = sqlite3.connect('recipes.db')
cur = conn.cursor()

# 2. 콘텐츠 기반 필터링 함수 (부분 문자열 매칭)
def recommend_recipes_by_ingredient(user_ingredient: str, top_n: int = 5):
    user_ingredient = user_ingredient.strip()
    # 레시피별로 재료를 모아서 가져옴
    cur.execute("""
        SELECT recipes.id, recipes.title, GROUP_CONCAT(ingredients.ingredient) as ingreds
        FROM recipes
        LEFT JOIN ingredients ON recipes.id = ingredients.recipe_id
        GROUP BY recipes.id
    """)
    rows = cur.fetchall()

    scored = []
    for recipe_id, title, ingreds_str in rows:
        if ingreds_str:
            ingred_list = [i.strip() for i in ingreds_str.split(',')]
            # 부분 문자열 매칭: 재료 중 하나라도 user_ingredient를 포함하면 점수 1
            match = any(user_ingredient in ing for ing in ingred_list)
            if match:
                scored.append((1, recipe_id, title))

    scored.sort(key=lambda x: x[0], reverse=True)
    return scored[:top_n]

# 3. 레시피 상세 출력 함수
def print_recipe_detail(recipe_id, title):
    print(f"\n[메뉴명] {title}")
    cur.execute('SELECT ingredient FROM ingredients WHERE recipe_id=?', (recipe_id,))
    ingredients = [row[0] for row in cur.fetchall()]
    print("[재료]", ', '.join(ingredients))
    print("[레시피]")
    cur.execute('SELECT step_order, description FROM steps WHERE recipe_id=? ORDER BY step_order', (recipe_id,))
    steps = cur.fetchall()
    for step_order, desc in steps:
        print(f"{step_order}. {desc}")

# 4. 사용 예시: "소고기" 입력
user_ingredient = "콩나물"
recommendations = recommend_recipes_by_ingredient(user_ingredient, top_n=5)

print(f"입력한 재료: {user_ingredient}")
if recommendations:
    for score, recipe_id, title in recommendations:
        print_recipe_detail(recipe_id, title)
else:
    print("해당 재료가 들어간 레시피가 없습니다.")

conn.close()

입력한 재료: 콩나물

[메뉴명] 국물이 너무 개운해요~맑은콩나물국 맛있게 끓이는법(김진옥요리가좋다)
[재료] 콩나물 4줌, 다진파 5T, 멸치다시마육수 500ml, 다진마늘 1/2T, 국간장 1T, 멸치액젓 1T, 소금 1/3T
[레시피]
1. 콩나물은 누런부분과 껍질부분 제거하고 두세번 세척후 소쿠리에 받쳐줍니다. 미리 멸치다시마육수 500ml도 준비해주세요~
2. 냄비에 씻은 콩나물을 넣고 멸치다시마육수 500ml를 넣은후 물 700ml를 넣고요~
3. 센불로 켠후 끓으면 중약불로 줄여 5분동안 끓여줍니다.
4. 5분경과후~ 다진마늘1/2T. 다진파5T. 국간장1T. 멸치액젓1T를 넣은후~ 한번 바글 끓인후 간을 보셔서 싱거우면 소금간을 추가하세요~ 저는 소금1/3T정도 넣었습니다.
